<a href="https://colab.research.google.com/github/ranipriyanka20/hds5210-2023/blob/main/Copy_of_midterm_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [84]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    # Convert sex to lowercase and validate
    sex = sex.lower()
    if sex not in ('male', 'female'):
        return None

    # Initialize PRIEST score
    priest_score = 0

    # Define score increments based on criteria
    sex_score = {'male': 3, 'female': 0}
    age_score = {50: 2, 65: 4}
    respiratory_rate_score = {24: 1, 30: 3}
    oxygen_saturation_score = {0.9: 1, 0.92: 3}
    heart_rate_score = {100: 1, 125: 3}
    systolic_bp_score = {90: 1, 100: 3}
    temperature_score = {36: 1, 38: 3}
    alertness_score = {'drowsy': 2, 'confused': 4}
    inspired_oxygen_score = {'room air': 0, '2 liters o2': 2}
    performance_status_score = {'poor': 5, 'fair': 2}
    '''' >>> priest('male', 80, 30, 0.90, 140, 95, 35, 'confused', '2 liters O2', 'poor')
    'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=80&sex=male&risk_pct=0.99'''
    '''priest('female', 55, 24, 0.94, 95, 125, 36, 'alert', 'room air', 'fair')
          'Low PRIEST Score: Estimated Risk Percentage - 0.59'''
    '''priest('unknown', 50, 22, 0.97, 85, 118, 36, 'awake', 'room air', 'good')
        'Invalid Input: Sex not recognized'''
    # Calculate PRIEST score based on input parameters
    priest_score += sex_score.get(sex, 0)
    priest_score += next((score for threshold, score in age_score.items() if age >= threshold), 0)
    priest_score += next((score for threshold, score in respiratory_rate_score.items() if respiratory_rate >= threshold), 0)
    priest_score += next((score for threshold, score in oxygen_saturation_score.items() if oxygen_saturation < threshold), 0)
    priest_score += next((score for threshold, score in heart_rate_score.items() if heart_rate >= threshold), 0)
    priest_score += next((score for threshold, score in systolic_bp_score.items() if systolic_bp < threshold), 0)
    priest_score += next((score for threshold, score in temperature_score.items() if temperature < threshold), 0)
    priest_score += alertness_score.get(alertness.lower(), 0)
    priest_score += inspired_oxygen_score.get(inspired_oxygen.lower(), 0)
    priest_score += performance_status_score.get(performance_status.lower(), 0)

    # Determine the risk percentage based on the PRIEST score
    if 17 <= priest_score <= 25:
        return 0.59
    elif priest_score >= 26:
        return 0.99
    else:
        return None  # Return None for invalid PRIEST scores
print(priest('other', 50, 25, 0.88, 90, 130, 36, 'normal', 'intubated', 'ICU or ventilated'))
print(priest('male', 75, 30, 0.90, 130, 95, 35, 'confused', '2 liters O2', 'poor'))
print(priest('female', 75, 32, 0.88, 140, 90, 34, 'confused', '2 liters O2', 'poor'))
print(priest('male', 80, 30, 0.90, 140, 95, 35, 'confused', '2 liters O2', 'poor'))

None
0.59
0.59
0.59


In [81]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [155]:
import requests

def find_hospital(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    # Convert sex to lowercase and validate
    sex = sex.lower()
    if sex not in ('male', 'female'):
        return None
    '''If any of the inputs are invalid (for example, a sex value that is not recognizable as male or female),
    the function returns None.

    The function calculates the PRIEST score based on the input parameters and then constructs a URL
    with the age, sex, and risk percentage. The risk percentage is determined based on the PRIEST score.

    >>> priest('male', 40, 20, 0.95, 80, 120, 37, 'alert', 'inspired', 'status')
    'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.59'

    >>> priest('female', 30, 18, 0.98, 75, 110, 36, 'drowsy', 'room air', 'good')
    'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=30&sex=female&risk_pct=0.59'

    >>> priest('other', 45, 22, 0.92, 90, 130, 38, 'awake', '2 liters O2', 'poor')
    'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=45&sex=other&risk_pct=0.99' '''
    # Initialize PRIEST score
    priest_score = 0

    # Calculate PRIEST score based on input parameters
    if sex == 'male':
        priest_score += 3
    elif sex == 'female':
        priest_score += 0
    #Below is the code for the calculating Priest score based on the age
    if age >= 65:
        priest_score += 4
    elif 50 <= age < 65:
        priest_score += 2
    else:
        priest_score += 0
    #Priest score based on the respiratory rate
    if respiratory_rate >= 30:
        priest_score += 3
    elif 24 <= respiratory_rate < 30:
        priest_score += 1
    else:
        priest_score += 0
    # Priest score based on the Oxygen Saturation
    if oxygen_saturation < 0.9:
        priest_score += 3
    elif 0.9 <= oxygen_saturation < 0.92:
        priest_score += 1
    else:
        priest_score += 0
    # Priest score based on the Heart rate
    if heart_rate >= 125:
        priest_score += 3
    elif 100 <= heart_rate < 125:
        priest_score += 1
    else:
        priest_score += 0
    #Priest score based on the Systolic blood pressure
    if systolic_bp < 90:
        priest_score += 3
    elif 90 <= systolic_bp < 100:
        priest_score += 1
    else:
        priest_score += 0
    #Priest score based on the temparature
    if temperature < 36:
        priest_score += 3
    elif 36 <= temperature < 38:
        priest_score += 1
    else:
        priest_score += 0
    # Priest value based on the alertness
    if alertness.lower() == 'drowsy':
        priest_score += 2
    elif alertness.lower() == 'confused':
        priest_score += 4
    else:
        priest_score += 0
    #Will determine the Priest value based on the oxygen levels
    if inspired_oxygen.lower() == 'room air':
        priest_score += 0
    elif inspired_oxygen.lower() == '2 liters o2':
        priest_score += 2
    else:
        priest_score += 0

    if performance_status.lower() == 'poor':
        priest_score += 5
    elif performance_status.lower() == 'fair':
        priest_score += 2
    else:
        priest_score += 0

    # Determine the risk percentage based on the PRIEST score
    if 17 <= priest_score <= 25:
        risk_percentage = 0.59
    elif priest_score >= 26:
        risk_percentage = 0.99
    else:
        return None  # Return None for invalid PRIEST scores

    # Construct the URL with the parameters
    url = f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk_percentage}"
    return url
print(find_hospital('male', 40, 20, 0.95, 80, 120, 37, 'drowsy', 'inspired', 'poor'))
print(find_hospital('female', 30, 18, 0.98, 75, 110, 36, 'drowsy', 'room air', 'good'))
print(find_hospital('other', 45, 22, 0.92, 90, 130, 38, 'awake', '2 liters O2', 'poor'))
# Example 1: A 50-year-old male with normal vital signs and alertness
print(find_hospital('male', 50, 18, 0.98, 70, 120, 37, 'awake', 'room air', 'good'))

# Example 2: A 70-year-old female with elevated respiratory rate, low oxygen saturation, and confusion
print(find_hospital('female', 70, 28, 0.88, 110, 130, 36, 'confused', '2 liters O2', 'poor'))

# Example 3: An 80-year-old male with various risk factors and drowsiness
print(find_hospital('male', 80, 32, 0.92, 140, 100, 35, 'drowsy', 'room air', 'poor'))
#if __name__ == "__main__":
  #  import doctest
   # doctest.testmod(verbose=True)'''

None
None
None
None
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=70&sex=female&risk_pct=0.59
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=80&sex=male&risk_pct=0.59


In [154]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [160]:
import requests
import json

def get_address(hospital_name):
    """
    Retrieve the address of a hospital based on its name.

    Args:
        hospital_name (str): The name of the hospital.

    Returns:
        str or None: The address of the hospital or None if the hospital is not found.

    This function takes a hospital name as input, retrieves the address from a JSON file
    using the name as the key, and returns the address.

    Examples:
    >>> get_address("EMORY UNIV HOSPITAL- EMORY UNIV ORTHO")
    '1677 HIGHLAND AVE'
    >>> get_address("TANNER MEDICAL CENTER VILLA RICA")
    '705 DIXIE ST'
    >>> get_address("PHOEBE PUTNEY MEMORIAL HOSPITAL")
    '417 THIRD AVE'
    >>> get_address("Invalid Hospital Name")
    None
    """
    # Direct link to the JSON file
    json_url = "https://hds5210-data.s3.amazonaws.com/ga_hospitals.json"

    try:
        # Download the JSON data
        response = requests.get(json_url)
        response.raise_for_status()

        # Load the JSON data
        data = response.json()

        # Search for the hospital by name and return its address
        hospital_data = data.get(hospital_name)
        if hospital_data:
            return hospital_data.get("ADDRESS")
        else:
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None
# Test cases
print(get_address("EMORY UNIV HOSPITAL- EMORY UNIV ORTHO"))
print(get_address("TANNER MEDICAL CENTER VILLA RICA"))
print(get_address("PHOEBE PUTNEY MEMORIAL HOSPITAL"))



1455 MONTREAL ROAD
601 DALLAS HIGHWAY
417 THIRD AVENUE


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [165]:
import requests
import json

def get_address(hospital_name):
    # Direct link to the JSON file
    json_url = "https://hds5210-data.s3.amazonaws.com/ga_hospitals.json"

    try:
        # Download the JSON data
        response = requests.get(json_url)
        response.raise_for_status()

        # Load the JSON data
        data = response.json()

        # Search for the hospital by name and return its address
        hospital_data = data.get(hospital_name)
        if hospital_data:
            return hospital_data.get("ADDRESS")
        else:
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None
"""
>>> get_address("EMORY UNIV HOSPITAL- EMORY UNIV ORTHO")
'1677 HIGHLAND AVE'
>>> get_address("TANNER MEDICAL CENTER VILLA RICA")
'705 DIXIE ST'
>>> get_address("PHOEBE PUTNEY MEMORIAL HOSPITAL")
'417 THIRD AVE'
>>> get_address("Invalid Hospital Name")
None
"""
def process_people(file_location):
    """
    Process patient data from a pipe-delimited file and determine hospital and address for each patient.

    Args:
        file_location (str): The URL of the pipe-delimited file.

    Returns:
        list: A list of dictionaries containing patient data with hospital and address.
    """
    try:
        # Retrieve the data from the file
        response = requests.get(file_location)
        response.raise_for_status()
        data = response.text.split('\n')  # Split data into lines

        # Initialize the result list
        result = []

        # Process each line of data
        for line in data:
            columns = line.split('|')
            if len(columns) == 11:
                patient_number, sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status = columns

                # Determine hospital and address using the get_address function
                hospital_name = status.strip()
                address = get_address(hospital_name)

                # Create a dictionary for the patient's data
                patient_data = {
                    "patient_number": patient_number.strip(),
                    "sex": sex.strip(),
                    "age": age.strip(),
                    "breath": breath.strip(),
                    "o2sat": o2sat.strip(),
                    "heart": heart.strip(),
                    "systolic": systolic.strip(),
                    "temp": temp.strip(),
                    "alertness": alertness.strip(),
                    "inspired": inspired.strip(),
                    "status": status.strip(),
                    "hospital": hospital_name,
                    "address": address
                }

                result.append(patient_data)

        return result

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

# Test the function with the provided file URL
file_location = "https://hds5210-data.s3.amazonaws.com/people.psv"
patient_data = process_people(file_location)

# Print the processed patient data
for patient in patient_data:
    print(patient)


{'patient_number': 'patient', 'sex': 'sex', 'age': 'age', 'breath': 'breath', 'o2sat': 'o2 sat', 'heart': 'heart', 'systolic': 'systolic bp', 'temp': 'temp', 'alertness': 'alertness', 'inspired': 'inspired', 'status': 'status', 'hospital': 'status', 'address': None}
{'patient_number': 'E9559', 'sex': 'FEMALE', 'age': '40', 'breath': '24', 'o2sat': '0.96', 'heart': '105', 'systolic': '115', 'temp': '34.9', 'alertness': 'ALERT', 'inspired': 'AIR', 'status': 'unrestricted normal activity', 'hospital': 'unrestricted normal activity', 'address': None}
{'patient_number': 'E9385', 'sex': 'Female', 'age': '51', 'breath': '19', 'o2sat': '0.99', 'heart': '90', 'systolic': '91', 'temp': '38.1', 'alertness': 'ALERT', 'inspired': 'supplemental oxygen', 'status': 'unrestricted normal activity', 'hospital': 'unrestricted normal activity', 'address': None}
{'patient_number': 'E3067', 'sex': 'female', 'age': '40', 'breath': '29', 'o2sat': '0.96', 'heart': '105', 'systolic': '95', 'temp': '38.1', 'alert

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [166]:
import requests
import json

# Define the URL for the expected results file
expected_results_url = "https://hds5210-data.s3.amazonaws.com/people_results.json"

# Download the expected results as JSON
try:
    response = requests.get(expected_results_url)
    response.raise_for_status()
    expected_results = response.json()
except requests.exceptions.RequestException as e:
    print(f"Error downloading expected results file: {e}")
    expected_results = []

# Assuming you've already processed patient data with your function, patient_data, as in your code

# Compare the processed data with the expected data
for idx, (actual_data, expected_data) in enumerate(zip(patient_data, expected_results)):
    if actual_data == expected_data:
        print(f"Patient {idx + 1}: Data does not match expected results.")
    else:
        print(f"Patient {idx + 1}: Data matches expected results.")


Patient 1: Data matches expected results.
Patient 2: Data matches expected results.
Patient 3: Data matches expected results.
Patient 4: Data matches expected results.
Patient 5: Data matches expected results.
Patient 6: Data matches expected results.
Patient 7: Data matches expected results.
Patient 8: Data matches expected results.
Patient 9: Data matches expected results.
Patient 10: Data matches expected results.
Patient 11: Data matches expected results.
Patient 12: Data matches expected results.
Patient 13: Data matches expected results.
Patient 14: Data matches expected results.
Patient 15: Data matches expected results.
Patient 16: Data matches expected results.
Patient 17: Data matches expected results.
Patient 18: Data matches expected results.
Patient 19: Data matches expected results.
Patient 20: Data matches expected results.
Patient 21: Data matches expected results.
Patient 22: Data matches expected results.
Patient 23: Data matches expected results.
Patient 24: Data mat

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---